In [3]:
import pandas as pd
import numpy as np
import sklearn as sk

#df = pd.read_csv("/Users/annesophieriis/Downloads/fatal-police-shootings-data.csv", sep=',', header=0)

df = pd.read_csv("/Users/leakrivaa/Downloads/DataMining-main-2/fatal-police-shootings-data.csv", sep=',', header=0)

#df = pd.read_csv("/Users/sofietosti/Desktop/DM Exam/fatal-police-shootings-data.csv", sep=',', header=0)

#See all rows in dataset
#pd.options.display.max_rows = None

In [4]:
df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True


In [5]:
df.dtypes
#maybe split date so we can sort per year?
#handle missing data in most columns
#maybe drop ' is-geocoding-exact'
#Drop name relevant? Names are not unique and GDPR reasons this will be removed?
#consider 'threat_level' - what does this mean?
#Flee can be made to boolean
#manner of death made to categorical - Maybe split this into how many have been  
#maybe use the longitude and latitude to make a heat map? where most deaths happen
#state, give numerical ID - categorical 
#armed Either boolean with armed and unarmed or categorical to see the how many have been armed with specific weapons.
#City to categorical - numeric ID 

id                           int64
name                        object
date                        object
manner_of_death             object
armed                       object
age                        float64
gender                      object
race                        object
city                        object
state                       object
signs_of_mental_illness       bool
threat_level                object
flee                        object
body_camera                   bool
longitude                  float64
latitude                   float64
is_geocoding_exact            bool
dtype: object

In [6]:
#Overview of content of coulmns 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5624 entries, 0 to 5623
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5624 non-null   int64  
 1   name                     5415 non-null   object 
 2   date                     5624 non-null   object 
 3   manner_of_death          5624 non-null   object 
 4   armed                    5412 non-null   object 
 5   age                      5372 non-null   float64
 6   gender                   5623 non-null   object 
 7   race                     5033 non-null   object 
 8   city                     5624 non-null   object 
 9   state                    5624 non-null   object 
 10  signs_of_mental_illness  5624 non-null   bool   
 11  threat_level             5624 non-null   object 
 12  flee                     5348 non-null   object 
 13  body_camera              5624 non-null   bool   
 14  longitude               

In [7]:
#Drop irrelevant coulmns 
df = df.drop(["name","is_geocoding_exact", "id", "longitude", "latitude"], axis = 1)

In [8]:
#Handling NaN values

#First looking at Armed. We assume that when NaN is a value in armed they must have been unarmed since there 
#hasn't been anything to report or seen. 

def handleArmed(df):
    df = df['armed'].fillna('unarmed', inplace = True)

handleArmed(df)

#Secondly, looking at Age, we replace the NaN values with the average age found with df.mean()

def handleAge(df):
    mean = df['age'].mean()

    df = df['age'].fillna(mean, inplace=True)
    
handleAge(df)

#Thirdly, we look at gender, where only 1 is missing a value, this will therefore be dropped.
def handleGender(df):
    df = df.dropna(subset=['gender'], inplace = True)

handleGender(df)

#Fourthly, we look at race which has around 500 missing values. These are considered important for our analysis
#Therefore, we decide to remove all rows that are missing a value.

def handleRace(df):
    df = df.dropna(subset=['race'], inplace = True)

handleRace(df)

def handleFlee(df):
    df = df.fillna('Not fleeing', inplace = True)
    #Making sure that there are only 4 different types. The remaining 227 are the NaN values. Hence,
    #most have the value not fleeing which is the one we will use to replace NaN values with. 
    """
    fleeList = df['flee'].tolist()
    countOne = 0
    countTwo = 0
    countThree = 0
    countFour = 0
    for f in fleeList:
        if(f == 'Not fleeing'):
            countOne += 1
        elif(f == 'Foot'):
            countTwo += 1
        elif(f == 'Car'):
            countThree += 1
        elif(f == 'Other'):
            countFour += 1
    
    print(countOne+countTwo+countThree+countFour)"""

handleFlee(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5032 entries, 0 to 5623
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     5032 non-null   object 
 1   manner_of_death          5032 non-null   object 
 2   armed                    5032 non-null   object 
 3   age                      5032 non-null   float64
 4   gender                   5032 non-null   object 
 5   race                     5032 non-null   object 
 6   city                     5032 non-null   object 
 7   state                    5032 non-null   object 
 8   signs_of_mental_illness  5032 non-null   bool   
 9   threat_level             5032 non-null   object 
 10  flee                     5032 non-null   object 
 11  body_camera              5032 non-null   bool   
dtypes: bool(2), float64(1), object(9)
memory usage: 442.3+ KB


In [10]:
# Reset index
df = df.reset_index()

In [12]:
"""
I want to split date into two columns, so we have one column only with year and one with full date
Date: Split so we have column with year ex. ["date"] 2015-01-02 -> 2015
"""
df['year'] = pd.DatetimeIndex(df['date']).year
#df.drop(['date'], axis=1)
df.head()

,index,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,year
0,0,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,2015
1,1,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,2015
2,2,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,2015
3,3,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,2015
4,4,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,2015


In [13]:
"""
Make manner_of_death of type categorial and encode values

0: shot
1: shot and Tasered

"""
df["manner_of_death"] = df["manner_of_death"].astype('category')
df["manner_of_death"] = df["manner_of_death"].cat.codes

In [14]:
"""
Make Armed of type categorical and encode values 
 
unmarmed: 0 (toy weapon, unarmed, pepper spray, undetermined)
armed: 1 (the rest)

NEW VERSION: LEA
"""
df['armed_encoded'] = ''

def addingArmedData(df):
    df["armed"] = df["armed"].astype('category')
    armedList = df['armed'].tolist()
    for x, val in enumerate(armedList, start=0):
        if val == "unarmed":
            armedList[x] = 0
        if val != "unarmed":
            armedList[x] = 1
    
    df["armed"] = armedList

addingArmedData(df)
df

,index,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,year,armed_encoded
0,0,2015-01-02,0,1,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,2015,
1,1,2015-01-02,0,1,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,2015,
2,2,2015-01-03,1,0,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,2015,
3,3,2015-01-04,0,1,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,2015,
4,4,2015-01-04,0,1,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,2015,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,5615,2020-09-04,0,1,33.0,M,B,Syracuse,NY,False,other,Not fleeing,True,2020,
5028,5616,2020-09-04,0,1,38.0,M,W,Lucerne,CA,False,attack,Not fleeing,False,2020,
5029,5617,2020-09-05,0,1,22.0,M,W,Battle Creek,MI,False,attack,Car,False,2020,
5030,5618,2020-09-05,0,1,61.0,M,B,San Antonio,TX,False,attack,Not fleeing,False,2020,


In [15]:
"""
0 = other and undetermined
1 = attack
"""
df['threatLevel_encoded'] = ''

def addingThreatLevelData(df):
    for index in df.index:
        if df.loc[index, 'threat_level'] == 'other':
            df.loc[df.index[index], 'threatLevel_encoded'] = 0

        
        elif df.loc[index, 'threat_level'] == 'undetermined':
            df.loc[df.index[index], 'threatLevel_encoded'] = 0
        
        else:
            df.loc[df.index[index], 'threatLevel_encoded'] = 1

addingThreatLevelData(df)
df

,index,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,year,armed_encoded,threatLevel_encoded
0,0,2015-01-02,0,1,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,2015,,1
1,1,2015-01-02,0,1,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,2015,,1
2,2,2015-01-03,1,0,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,2015,,0
3,3,2015-01-04,0,1,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,2015,,1
4,4,2015-01-04,0,1,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,2015,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,5615,2020-09-04,0,1,33.0,M,B,Syracuse,NY,False,other,Not fleeing,True,2020,,0
5028,5616,2020-09-04,0,1,38.0,M,W,Lucerne,CA,False,attack,Not fleeing,False,2020,,1
5029,5617,2020-09-05,0,1,22.0,M,W,Battle Creek,MI,False,attack,Car,False,2020,,1
5030,5618,2020-09-05,0,1,61.0,M,B,San Antonio,TX,False,attack,Not fleeing,False,2020,,1


In [16]:
"""

Make gender categorical and encode values, random order.

M / male = 1
F / female = 0

"""

def genderCat(df):
    df["gender"] = df["gender"].astype('category')
    df["gender"] = df["gender"].cat.codes
    df

genderCat(df)

In [17]:
"""

Make age type int to remove decimals.

"""
def convertAge(df):
    df.age = df.age.apply(int)

convertAge(df)

In [18]:
"""
Changing object data types to categorical:
A = 0 
B = 1
H = 2
N = 3
0 = 4
W = 5
"""
df["race"] = df["race"].astype('category')
df["race"] = df["race"].cat.codes

df

,index,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,year,armed_encoded,threatLevel_encoded
0,0,2015-01-02,0,1,53,1,0,Shelton,WA,True,attack,Not fleeing,False,2015,,1
1,1,2015-01-02,0,1,47,1,5,Aloha,OR,False,attack,Not fleeing,False,2015,,1
2,2,2015-01-03,1,0,23,1,2,Wichita,KS,False,other,Not fleeing,False,2015,,0
3,3,2015-01-04,0,1,32,1,5,San Francisco,CA,True,attack,Not fleeing,False,2015,,1
4,4,2015-01-04,0,1,39,1,2,Evans,CO,False,attack,Not fleeing,False,2015,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,5615,2020-09-04,0,1,33,1,1,Syracuse,NY,False,other,Not fleeing,True,2020,,0
5028,5616,2020-09-04,0,1,38,1,5,Lucerne,CA,False,attack,Not fleeing,False,2020,,1
5029,5617,2020-09-05,0,1,22,1,5,Battle Creek,MI,False,attack,Car,False,2020,,1
5030,5618,2020-09-05,0,1,61,1,1,San Antonio,TX,False,attack,Not fleeing,False,2020,,1


In [19]:
"""
Making state to categorical value giving each state an ID
Can only be run once since the coulm 
"""
def transformState(df):
    df["state"] = df["state"].astype('category')
    stateIDvalue = df["state"].cat.codes
    df.insert(loc=8, column='state_ID', value = stateIDvalue)

transformState(df)


In [20]:
#creating two new features consisting of a boolean indicating if False = not fleeing and True = fleeing
#And flee_encoded which is either 0 for not fleeing or 1 for fleeing

#df['flee_boolean'] = False
df['flee_encoded'] = '' 

def addingFleeData(df):
    for index in df.index:
        if df.loc[index, 'flee'] == 'Not fleeing':
            #df.loc[df.index[index], 'flee_boolean'] = False
            df.loc[df.index[index], 'flee_encoded'] = 0
            
        elif df.loc[index, 'flee'] != 'Not fleeing':
            #df.loc[df.index[index], 'flee_boolean'] = True
            df.loc[df.index[index], 'flee_encoded'] = 1

addingFleeData(df)

#making flee_encoded to category
df["flee_encoded"] = df["flee_encoded"].astype('category')
df["flee_encoded"] = df["flee_encoded"].cat.codes

In [23]:
"""
Moving the columns so they are next to the ones they are related to.
"""
column_names = ["index", "date", "year","manner_of_death","armed","age","gender","race","city","state","state_ID","signs_of_mental_illness","threat_level","threatLevel_encoded","flee","flee_encoded","body_camera"]
df = df.reindex(columns=column_names)

clean_df = df

In [24]:
clean_df

,index,date,year,manner_of_death,armed,age,gender,race,city,state,state_ID,signs_of_mental_illness,threat_level,threatLevel_encoded,flee,flee_encoded,body_camera
0,0,2015-01-02,2015,0,1,53,1,0,Shelton,WA,47,True,attack,1,Not fleeing,0,False
1,1,2015-01-02,2015,0,1,47,1,5,Aloha,OR,37,False,attack,1,Not fleeing,0,False
2,2,2015-01-03,2015,1,0,23,1,2,Wichita,KS,16,False,other,0,Not fleeing,0,False
3,3,2015-01-04,2015,0,1,32,1,5,San Francisco,CA,4,True,attack,1,Not fleeing,0,False
4,4,2015-01-04,2015,0,1,39,1,2,Evans,CO,5,False,attack,1,Not fleeing,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,5615,2020-09-04,2020,0,1,33,1,1,Syracuse,NY,34,False,other,0,Not fleeing,0,True
5028,5616,2020-09-04,2020,0,1,38,1,5,Lucerne,CA,4,False,attack,1,Not fleeing,0,False
5029,5617,2020-09-05,2020,0,1,22,1,5,Battle Creek,MI,22,False,attack,1,Car,1,False
5030,5618,2020-09-05,2020,0,1,61,1,1,San Antonio,TX,43,False,attack,1,Not fleeing,0,False


In [ ]:
clean_df.to_csv("clean_data.csv", index = False)

In [ ]:
clean_df